# <center><font size="8">Prototipo de Oferta Dinámica Personalizada</font></center>

<div align="left"><font size="5">📌 Introducción</font></div>

## 1. Contexto y Objetivos

El presente proyecto se enfoca en el desarrollo de un sistema avanzado de **Oferta Dinámica (Dynamic Pricing)** con el objetivo primordial de **optimizar los ingresos y la rentabilidad** a través de la **personalización de tarifas**.  

En lugar de utilizar modelos de precio estáticos o ajustes basados únicamente en la demanda global, nuestro prototipo se cimenta en el **análisis individual del consumidor** para determinar el precio óptimo en tiempo real.  

Para lograr esta personalización, hemos diseñado un **pipeline de Machine Learning** que combina técnicas de **econometría**, **aprendizaje no supervisado** y **modelos predictivos**.

---

## 2. Metodología y Arquitectura del Modelo

La propuesta metodológica se basa en **tres fases interconectadas**, que transforman la sensibilidad del cliente en una tarifa personalizada:

---

### A. Estimación y Segmentación por Elasticidad (Fundamento Económico)

El primer pilar del prototipo fue la **estimación rigurosa de la elasticidad precio de la demanda ($\text{EPD}$)**.  
Esta métrica ha sido fundamental para entender el grado de respuesta de la cantidad demandada ante variaciones de precio en nuestros datos históricos.

Posteriormente, aplicamos un **algoritmo de clustering** (*aprendizaje no supervisado*) para **clasificar a los clientes** basándonos en su $\text{EPD}$ y otras características de comportamiento.  

Este *clustering* nos permitió crear **segmentos de clientes bien definidos** (por ejemplo: *“Extremadamente Elásticos”*, *“Inelásticos”*), estableciendo un **rango de ajuste de precios** (aumento o decremento) específico para cada grupo.

---

### B. Clasificación de Clientes Nuevos (Motor de XGBoost)

Una vez establecidos los segmentos de elasticidad, desarrollamos un **clasificador basado en el algoritmo XGBoost (eXtreme Gradient Boosting)**.  

La función de este modelo es crucial en la operativa:  
al recibir los datos de un nuevo cliente o una nueva interacción, **XGBoost predice a cuál de las categorías de elasticidad pre-clasificadas pertenece**.  

Esta predicción es el **factor determinante inicial** para estimar la tarifa personalizada, permitiéndonos saber si el sistema debe apuntar a un **aumento**, un **decremento** o un **precio base**.

---

### C. Determinación de la Oferta Dinámica (Red Neuronal)

Finalmente, para la etapa de determinación del precio, implementamos una **red neuronal artificial**.  

Este modelo de aprendizaje profundo **integra las características de la demanda**, la **clasificación de elasticidad obtenida de XGBoost**, y otras **variables dinámicas** (como inventario, hora del día y actividad de la competencia) para **estimar la oferta dinámica final y personalizada**.  

La red neuronal provee la **flexibilidad** y la **capacidad de capturar relaciones no lineales** necesarias para el ajuste preciso del precio a nivel individual.



<div align="left"><font size="5">📌 Objetivos</font></div>


- **1. Estimar la Elasticidad de la Demanda**  
  - Cuantificar la sensibilidad al precio ($\text{EPD}$) de los diferentes segmentos de clientes.  
  - Servirá como fundamento económico para la toma de decisiones de precio.

- **2. Segmentar Clientes por Comportamiento de Precio**  
  - Aplicar técnicas de *clustering para agrupar clientes en categorías de elasticidad homogéneas.  
  - Crear segmentos accionables que definan la estrategia inicial de aumento o decremento de precios.

- **3. Desarrollar un Motor de Clasificación de Clientes**  
  - Implementar y validar el algoritmo XGBoost para clasificar con alta precisión a nuevos clientes.  
  - Integrar la clasificación dentro de los segmentos de elasticidad previamente definidos.

- **4. Integrar un Pipeline Predictivo Completo**  
  - Crear un prototipo pipeline robusto que combine:  
    - **Clustering** (aprendizaje no supervisado)  
    - **Clasificación** (XGBoost)  
    - **Predicción** (Red Neuronal)  
- **5. Generar la tarifa final personalizada con tarifa dinámica**.



# <div align="left"><font size="5">📌 Librerias</font></div>

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

from Tools import Tools4DataExtraction as T4DE
from Tools.Tools4Cluster import ClusteringData
from Tools.Tools4ClasSupervisada import ClusteringSupervisado
from Tools import Tools4TarifPer as T4TP
from Tools import Tools4Net as T4N 
from Tools import Tools4Elasticity as T4E
from Tools import Tools4CaracDemanda as TCD

# <div align="left"><font size="5">📌Extracción de información del modelo de datos</font></div>

In [2]:
# Se extraen los datos del modelo de datos
D4NN, D4C, D4C1, D4GC= T4DE.Get_Data()

Memoria usada antes: 626.74 MB
Memoria usada después: 121.78 MB
Reducción: 80.6%


In [3]:
TCD.BuenasCaracteristicas(D4GC)

# <div align="left"><font size="5">📌Calculo de la elasticidad</font></div>

La Elasticidad Precio de la Demanda (EPD) mide cómo la cantidad demandada de un producto cambia en respuesta a una variación en su precio. Se calcula como 

$$\epsilon_{Q} = \frac{\Delta Q}{\Delta P}$$

### Clasificación de la Elasticidad de la Demanda según su valor

**Demanda Elástica** ($|EPD| > 1$):  
La cantidad demandada varía en una proporción mayor que el cambio en el precio.  
Un pequeño cambio en el precio provoca un cambio significativo en la cantidad demandada.  
*Ejemplo:* Bienes de lujo o productos con muchos sustitutos disponibles.

---

**Demanda Inelástica** ($|EPD| < 1$):  
La cantidad demandada varía en una proporción menor que el cambio en el precio.  
Un cambio en el precio tiene poco impacto en la cantidad demandada.  
*Ejemplo:* Bienes de primera necesidad o esenciales, como medicamentos o gasolina (a corto plazo).

---

**Elasticidad Unitaria** ($|EPD| = 1$):  
La cantidad demandada varía en la misma proporción que el cambio en el precio.

In [4]:
T4E.MainElas()

Memoria usada antes: 626.74 MB
Memoria usada después: 121.78 MB
Reducción: 80.6%
La elasticidad de la demanda es: 0.7756


# <div align="left"><font size="5">📌Clasificación no supervisada</font></div>

La clasificación de clientes permite entender cómo distintos grupos responden al precio y al servicio, habilitando estrategias de precio personalizado que maximizan ingresos y reducen pérdida de demanda. Se calcula el **precio óptimo** según la respuesta esperada del segmento (elasticidad estimada o probabilidad de compra).

### 🔹 Estrategias diferenciadas
- **Segmento elástico →** precios competitivos o descuentos.  
- **Segmento inelástico →** precios premium o aumentos graduales.  
- **Segmento leal →** estabilidad de precios.

---

## 📊 Beneficios Clave

- 💰 **Maximiza los ingresos** ajustando precios según la disposición a pagar.  
- 📉 **Reduce pérdida de demanda** por aumentos injustificados.  
- 🎯 **Mejora la precisión del modelo**, al trabajar con segmentos más homogéneos.  
- 🧠 **Permite estrategias personalizadas y accionables** según el tipo de cliente.


In [5]:
ClusteringData(False,D4C)

Modelo K-Means guardado exitosamente como: KmeansAllData.joblib


### Estrategia de Precios por *Cluster*

---

#### 1. Incrementar Precios (o Reducir Descuentos) 📈  
**Objetivo:** Maximizar el margen en clientes de alto valor.  

| **Cluster** | **Acción** | **Razón Clave** |
|--------------|-------------|-----------------|
| **2** | Aumentar el precio. | **Élite:** Máximo gasto  y máxima actividad . Demanda inelástica. |
| **0** | Evaluar aumento moderado. | **Alto Valor:** Alto gasto. Monitorear para no perderlos por su menor actividad. |

---

#### 2. Bajar Precios (o Aumentar Promociones) 📉  
**Objetivo:** Impulsar el volumen y reactivar a clientes sensibles al precio o en riesgo.  

| **Cluster** | **Acción** | **Razón Clave** |
|--------------|-------------|-----------------|
| **5** | Aumentar las promociones. | **Sensibles:** Bajo valor y alta indicación de sensibilidad a promociones. |
| **3** | Ofertas de reactivación. | **En Riesgo:** Gasto inicial alto, pero actividad posterior nula. Necesitan un incentivo fuerte para reengancharse. |

---

#### 3. Mantener Precios (Enfoque en Volumen y Venta Cruzada) 🛍️  
**Objetivo:** Fidelizar y aumentar la compra de productos adicionales sin tocar la base del precio.  

| **Cluster** | **Acción** | **Razón Clave** |
|--------------|-------------|-----------------|
| **1 y 4** | Mantener precios. | **Muy Activos:** Ya tienen una alta frecuencia de compra. La meta es venderles más productos, no bajarles ni bajarles el precio base. |


# <div align="left"><font size="5">📌Extracción de la base de datos de los clusters generados</font></div>

In [6]:
# se extrae la base de datos de los clusters generados
DBClus= T4DE.GetDB()

# <div align="left"><font size="5">📌Clasificación supervisada</font></div>


La implementación de un algoritmo de aprendizaje supervisado es fundamental para transformar una base de datos de clientes clasificados en un sistema de tarifa dinámica eficiente y rentable.

El valor agregado reside en pasar de la segmentación estática a la predicción continua. 

| Componente | Rol en el Modelo de Tarifa Dinámica | Beneficio Clave |
| :--- | :--- | :--- |
| **Clasificación Inicial de Clientes** | Sirve como *input* (una de las características) para el modelo de Machine Learning. | Proporciona una base de datos histórica y segmentada de alta calidad. |
| **Algoritmo de Aprendizaje Supervisado** | Entrena un modelo para predecir el comportamiento del cliente y calcular el precio óptimo basándose en datos históricos etiquetados (transacciones y resultados). | Maximiza el ingreso al ajustar la tarifa en tiempo real a la disposición a pagar individual. |
| **Red neuronal (Output)** | Permite al modelo predecir un valor continuo y exacto (el precio de venta) en lugar de limitarse a asignar categorías predefinidas. | Permite un ajuste fino y automatizado de precios, respondiendo a la demanda, inventario, precios de la competencia y el perfil específico del cliente simultáneamente. |



In [7]:
# Se aplica la clasificación supervisada a la base de datos de los clusters generados
ClusteringSupervisado(DBClus)

Modelo XGBoost guardado exitosamente como: modelo_xgboost_clientes.json


# <div align="left"><font size="5">📌Calculando la tarifa dinámica</font></div>

Una red neuronal artificial (RNA) es una herramienta poderosa para pronosticar la tarifa dinámica porque sobresale en el manejo de la complejidad y las relaciones no lineales inherentes a la fijación de precios en el mercado.

Su principal utilidad es su capacidad para modelar la elasticidad de la demanda de forma muy precisa, considerando simultáneamente una gran cantidad de variables.

In [8]:
# 0 es para todos los dias antes de ayer
# -1 es para todos los dias antes de 6 dias desde ayer
# 1 es desde ayer hasta hace un año
T4N.ProcessingNet(D4NN,2)

# para pronosticar con todo el dataframe

# False es para el dia de hoy
# True es para hace 6 dias hasta hoy
df_= T4N.PrepareData4Fore(D4NN,False)

#Fore= T4N.GetTodayData4Net(df_)
#PrecioDin=T4N.NewClientsPredNet(Fore)
#df_['PRECIO DINAMICO']=PrecioDin
#TodayDataPD= df_.copy()

# para pronosticar por cliente el dataframe
lista_resultados = []

for row in range(len(df_)):
    InfoClient = pd.DataFrame(df_.iloc[row]).T
    Foree= T4N.GetTodayData4Net(InfoClient)
    InfoClient["TARIFA DINAMICA"] = T4N.NewClientsPredNet(Foree)[0,0]
    
    lista_resultados.append(InfoClient)

TodayDataPD = pd.concat(lista_resultados, ignore_index=True)

Epoch 1/2
6952/6952 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - loss: 26442.5879 - mae: 54.3419 - mse: 26442.5879 - val_loss: 216.6147 - val_mae: 10.5056 - val_mse: 216.6147
Epoch 2/2
6952/6952 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - loss: 189.5386 - mae: 9.6167 - mse: 189.5386 - val_loss: 165.8195 - val_mae: 8.7707 - val_mse: 165.8195
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 1s 476us/step

El Error Absoluto Medio (MAE) final es de: 8.76 [Moneda]

La Raíz del Error Cuadrático Medio (RMSE) final es de: 12.80 [Moneda]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━

# <div align="left"><font size="5">📌Obteniendo la elasticidad calculada</font></div>

La elasticidad calculada se guarda en un modelo de datos para posteriormente cargarla y hacer los cálculos necesarios en la segmentación de los clientes.

In [9]:
DataElas= T4E.GetDataElasticity()
Elas=DataElas['Elasticidad']

# <div align="left"><font size="5">📌Obteniendo datos del presente día para la tarifa personalizada</font></div>

Se cargan los datos del día presente del modelo de datos

In [10]:
# Se obtienen los datos de los clientes para obtener la tarifa personalizada
TodayData4C= T4TP.GetTodayData4Cluster(D4C1)

# <div align="left"><font size="5">📌Calculando el porcentaje de aumento para la tarifa personalizada</font></div>

Ya teniendo entrenados los algoritmos que clasificaron la data histórica de clientes, de manera supervisada y no supervisada, se procede a realizar una prueba de calidad con los datos del día corriente para poder observar el comportamiento de los precios personalizados.

In [11]:
# se simula la obtencion del cluster del cliente que está comprando
# para ver que tarifa personalizada se le asigna

import pandas as pd

# 1. Inicializa una lista vacía para guardar los resultados
lista_resultados = []

# 2. Itera y agrega cada DataFrame a la lista
for row in range(len(TodayData4C)):
    # 1. Crea la fila (DataFrame de 1xN)
    InfoClient = pd.DataFrame(TodayData4C.iloc[row]).T
    
    # 2. Procesa la información (asumo que T4TP.GetCluster devuelve Cluster y desc)
    Cluster, desc = T4TP.GetCluster(InfoClient, DBClus,Elas)
    
    # 3. Agrega la nueva columna
    InfoClient["%_Tarifa Personalizada"] = desc
    InfoClient["Cluster"] = Cluster
    InfoClient["Delta_Q"] = desc*Elas
    
    # 4. Agrega el DataFrame resultante a la lista
    lista_resultados.append(InfoClient)

# 3. Concatena todos los DataFrames de la lista de una sola vez
TodayDataTP = pd.concat(lista_resultados, ignore_index=True)

# El DataFrame 'final' ahora contiene todas las filas concatenadas

# <div align="left"><font size="5">📌Merge de las tarifas</font></div>

Para finalizar este pipeline, se combinan los dataframes obtenidos de la red neuronal del precio dinámico con el del algoritmo de clasificación del precio personalizado

In [12]:
df_combinado = pd.merge(
    TodayDataPD,
    TodayDataTP,
    on='NOMBRE_PASAJERO',  # La columna clave que comparten
    how='inner'    # El tipo de unión que deseas
)

In [13]:
df_combinado.columns

Index(['NOMBRE_PASAJERO', 'BOLETOS_VEND_x', 'FECHA_CORRIDA_x',
       'HORA_SALIDA_CORRIDA', 'CLASE_SERVICIO_x', 'IVA_TARIFA_BASE_TRAMO',
       'PAX_SUBEN', 'TARIFA_BASE_TRAMO', 'FECHA_OPERACION_x', 'HORA_OPERACION',
       'VENTA_x', 'DISPONIBILIDAD_TRAMO', 'HORAS_ANTICIPACION_x', 'ORIGEN',
       'DESTINO', 'TIPO_CLIENTE', 'NUM_ASIENTO', 'TARIFA DINAMICA',
       'BOLETOS_VEND_y', 'CLASE_SERVICIO_y', 'DESC_DESCUENTO',
       'DIAS_ANTICIPACION', 'EMAIL', 'FECHA_CORRIDA_y', 'FECHA_OPERACION_y',
       'HORAS_ANTICIPACION_y', 'PAGO_METODO', 'PORCENT_PROMO', 'TIPO_CORRIDA',
       'TIPO_PASAJERO', 'TOTAL_BOLETOS', 'VENTA_y', 'VENTA_ANTICIPADA',
       'VENTA_TOTAL', '%_Tarifa Personalizada', 'Cluster', 'Delta_Q'],
      dtype='object')

In [14]:
df_PD= df_combinado[['NOMBRE_PASAJERO', 'EMAIL','Cluster', 'TARIFA DINAMICA', '%_Tarifa Personalizada','TARIFA_BASE_TRAMO',"Delta_Q"]]
df_PD ['Precio final con IVA']= df_PD['TARIFA DINAMICA']*(df_PD['%_Tarifa Personalizada']/100)+ df_PD['TARIFA DINAMICA']+df_PD['TARIFA DINAMICA']*0.16
df_PD ['Precio final con IVA']= np.round(df_PD ['Precio final con IVA'])

df_PD ['Precio final sin IVA']= df_PD['TARIFA DINAMICA']*(df_PD['%_Tarifa Personalizada']/100)+ df_PD['TARIFA DINAMICA']
df_PD ['Precio final sin IVA']= np.round(df_PD ['Precio final sin IVA'])

El siguiente dataframe muestra los precios dinámicos con el 15 porciento de IVA para cada uno de los usuarios del presente día según su clasificación obtenida con los algoritmos de aprendizaje. 

In [15]:
df_PD

NOMBRE_PASAJERO  \
0    ALEJANDRO OMAR MIJANGOS JURADO   
1                SAUL NIZ BALVANEDA   
2            SALVADOR CHAVEZ VARGAS   
3            ALEJANDRO MATA MORALES   
4            RICARDO GARAY ESCOBEDO   
5           COMPLENTO ULISES CHAVEZ   
6    MARIA DEL ROCIO BARBA ESPINOZA   
7         JOSÉ RAÚL NAVARRO CAREAGA   
8                   DANIELA MIRANDA   
9             ALFONSO RAMIREZ MUÑOZ   
10                ELISA MEZA TORRES   
11              EFRAI SALADO MARQEZ   
12       ARTURO VILLALOBOS GUERRERO   
13          MARTIN BERNARDINO LOPEZ   
14           ARTURO VILLEGAS MACIAS   
15     LAYLA WEISENBERGER MC EVILLY   
16   IVETTE ANABELL YAÑEZ MALDONADO   
17              VICTOR MONROY OMAÑA   
18         ADRIANA CHAVARRIA GARCIA   
19  PEDRO ALONSO RUVALCABA GONZALEZ   
20    PEDRO ALONSO RUVALCABA RIVERA   
21           SANTIAGO MAISTERRA BRU   
22        JUAN PABLO ARAIZA JIMENEZ   
23     KEVIN SALVADOR VELASCO ORTIZ   
24         HUGO OSIRIS OCHOA CORTES   
25     JESUS MARTIN BRIONES AGUILAR   

                                          EMAIL  Cluster  TARIFA DINAMICA  \
0    alejandro.omar.mijangos.jurado@ejemplo.com        1      1156.197266   
1                saul.niz.balvaneda@ejemplo.com        1      1162.873291   
2            salvador.chavez.vargas@ejemplo.com        5      1162.673950   
3            alejandro.mata.morales@ejemplo.com        1      1166.994385   
4            ricardo.garay.escobedo@ejemplo.com        5      1164.208252   
5           complento.ulises.chavez@ejemplo.com        5      1163.678101   
6    maria.del.rocio.barba.espinoza@ejemplo.com        3      1161.572388   
7         josé.raúl.navarro.careaga@ejemplo.com        5      1162.973511   
8                   daniela.miranda@ejemplo.com        3      1161.974121   
9             alfonso.ramirez.muñoz@ejemplo.com        3      1162.579102   
10                elisa.meza.torres@ejemplo.com        3      1162.980957   
11              efrai.salado.marqez@ejemplo.com        3      1165.833740   
12       arturo.villalobos.guerrero@ejemplo.com        3      1163.482910   
13          martin.bernardino.lopez@ejemplo.com        3      1162.077393   
14           arturo.villegas.macias@ejemplo.com        5      1164.814819   
15     layla.weisenberger.mc.evilly@ejemplo.com        5      1164.919312   
16   ivette.anabell.yañez.maldonado@ejemplo.com        5      1164.461182   
17              victor.monroy.omaña@ejemplo.com        5      1169.741943   
18         adriana.chavarria.garcia@ejemplo.com        5      1164.497559   
19  pedro.alonso.ruvalcaba.gonzalez@ejemplo.com        5      1164.679932   
20    pedro.alonso.ruvalcaba.rivera@ejemplo.com        5      1164.619629   
21           santiago.maisterra.bru@ejemplo.com        5      1170.660400   
22        juan.pablo.araiza.jimenez@ejemplo.com        3      1165.526245   
23     kevin.salvador.velasco.ortiz@ejemplo.com        3      1164.430664   
24         hugo.osiris.ochoa.cortes@ejemplo.com        5      1169.525146   
25     jesus.martin.briones.aguilar@ejemplo.com        5      1168.990356   

    %_Tarifa Personalizada TARIFA_BASE_TRAMO   Delta_Q  Precio final con IVA  \
0                      0.0       1163.790039  0.000000                1341.0   
1                      0.0       1163.790039  0.000000                1349.0   
2                      2.0       1163.790039  1.551149                1372.0   
3                      0.0       1163.790039  0.000000                1354.0   
4                      2.0       1163.790039  1.551149                1374.0   
5                      2.0       1163.790039  1.551149                1373.0   
6                      2.0       1163.790039  1.551149                1371.0   
7                      2.0       1163.790039  1.551149                1372.0   
8                      2.0       1163.790039  1.551149                1371.0   
9                      2.0       1163.790039  1.551149                1372.0   
10    

In [16]:
# solo para el final del dia
df_PD.to_csv('datos_dinamicos.csv', index=False)

In [17]:
ruta_principal = os.getcwd()
config_path = os.path.join(ruta_principal, "Files", 'datos_dinamicos.parquet')
df_PD.to_parquet(config_path, index=False)
config_path = os.path.join(ruta_principal, "Files", 'datos_dinamicos.csv')
df_PD.to_csv(config_path, index=False)